# Kaggle Salt Segmentation - Create model from pre-trained based on resnet

Link to competition: https://www.kaggle.com/c/tgs-salt-identification-challenge

This notebook was converted from my prior Kaggle notebook.  Migrated to TF 2.x and converted various methods to be more native TF. This will create a model from a pre-trained model.

- Pre-trained model is from Pavel Yakubovshiy, (https://github.com/qubvel/segmentation_models) 
- Images are png
- Doubled the number of training images to give additional examples.  Augmentation was applied to all images to reduce the impact of having duplicates.
- Training took about 10 epocs, final metrics were: loss: 0.0249 - lb_metric: 0.8959 - val_loss: 0.1496 - val_lb_metric: 0.8103
- Validation scores from Training images: (224, 224) Score:  0.82    (101, 101) Score:  0.79
- Running on Google Colab took about 15 mins

 (224, 224) Score:  0.8135483863372956    (101, 101) Score:  0.791756271007454

In [0]:
# Comment out if not using Google Colab

#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
%tensorflow_version 2.x
#"""

### Start Kaggle Data Download

In [0]:
# Upload your "kaggle.json" file that you created from your Kaggle Account tab
# If you downloaded it, it would be in your "Downloads" directory
from google.colab import files
files.upload()

In [0]:
# Double check to see what files already exist
!ls

In [0]:
# On your VM, create kaggle directory and modify access rights
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
# Install kaggle libs
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

In [0]:
# Download salt files and unzip
!kaggle competitions download -c tgs-salt-identification-challenge
!ls
!unzip -q tgs-salt-identification-challenge.zip
!ls
!unzip -q train.zip -d train
!ls 

### End Kaggle Data Install, Start Normal Notebook

In [0]:
# Setup sys.path to find MachineLearning lib directory

try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [0]:
# Normal includes...
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv
import numpy as np 

import IPython.display as display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.models import load_model 

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# custom libraries
from TrainingUtils import *
from losses_and_metrics.Losses_Babakhin import make_loss, Kaggle_IoU_Precision, dice_coef_loss_bce

# This allows the runtime to decide how best to optimize CPU/GPU usage
AUTOTUNE = tf.data.experimental.AUTOTUNE
model = None

## Various Methods

In [0]:
# Set these to match your environment

if USING_COLLAB:
    ROOT_PATH = ""
    MODEL_PATH = "/content/drive/My Drive/ImageData/KaggleSaltDeposits/"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/KaggleSaltDeposits/"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    MODEL_PATH = None

# Establish global dictionary
parms = GlobalParms(ROOT_PATH=ROOT_PATH,
                    MODEL_NAME="salt_segmodel-224-V01.h5",
                    TRAIN_DIR="train", 
                    MODEL_PATH=MODEL_PATH,
                    NUM_CLASSES=1,
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=25,
                    EPOCS=30,
                    IMAGE_EXT=".png")
# other globals
STARTING_MODEL_PATH = "/content/drive/My Drive/GitHub/MachineLearning/2-KaggleSalt/segmodel-224-224-c1-V01.h5"

parms.print_contents()

In [0]:
# Helper method to display images and masks
def show_batch_mask(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        #print(np.max(display_list[i]), np.min(display_list[i]))
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]),cmap='gray')
        plt.axis('off')
    plt.show()

## Create training and validation files

The number of ships is not balanced and the size of some of the images are very small.  The approach can be changed if you want.  

In [0]:
# Load files and create dataframe
image_path = os.path.join(parms.TRAIN_PATH, "images")
all_files_tmp = np.array(os.listdir(image_path))
all_files = []
for image_id in all_files_tmp:
    if image_id.endswith(parms.IMAGE_EXT):
        all_files.append(image_id)

print("All files: ", len(all_files), " ", all_files[0])

# modify to reduce the number of images processed, or comment out for full list
all_files = all_files

# Create df from files
all_df = pd.DataFrame(all_files, columns =['ImageId']) 
all_df.head()

In [0]:
masks_path = os.path.join(parms.TRAIN_PATH, "masks")
images_path = os.path.join(parms.TRAIN_PATH, "images")
 
# Load mask size and use to guess about salt
all_df['mask_size'] = all_df['ImageId'].map(lambda image_id: round(os.stat(os.path.join(masks_path, image_id)).st_size))
all_df['has_salt'] = all_df['mask_size'].map(lambda x: 0 if x < 90 else 1)

# load image size and guess about all black image
all_df["image_size"] = all_df['ImageId'].map(lambda image_id: round(os.stat(os.path.join(images_path, image_id)).st_size) )
all_df["black_image"] = all_df['image_size'].map(lambda x: 1 if x < 110 else 0)

# Delete all black images
blackimages = all_df[all_df['black_image'] == 1].index
all_df.drop(blackimages , inplace=True)

all_df.describe()

In [0]:
# Create more training images, will reduce later
all_df = pd.concat([all_df, all_df])
all_df = shuffle(all_df)

In [0]:
# Stratifing by image_size, my prior notebook used the number of white pixels, this was easier and gave a better spread
all_df_cut = pd.cut(all_df["image_size"], bins=[0, 8000, 9000, 10250, 12000, 100000]) 
ax = all_df_cut.value_counts(sort=False).plot.bar(rot=0, color="b", figsize=(20,6)) 
plt.show() 

In [0]:
# Apply method to create the group number
def group_by_image_size(x):
    #[0, 8000, 9000, 10250, 12000, 100000]) 
    if x < 8000:
        return 0
    elif x < 9000:
        return 1
    elif x < 10250:
        return 2
    elif x < 12000:
        return 3
    else:
        return 4

all_df['image_group'] = all_df['image_size'].apply(group_by_image_size)
all_df.head()


In [0]:
# Select a balanced subset for training
SAMPLES_PER_GROUP = 1400
balanced_train_df = all_df.groupby('image_group').apply(lambda x: x.sample(SAMPLES_PER_GROUP) if len(x) > SAMPLES_PER_GROUP else x)
balanced_train_df['image_group'].hist(bins=balanced_train_df['image_group'].max()+1)
print(balanced_train_df.shape[0], 'image_group')


In [0]:
# Create training and validation lists from the balanced df

train_df, valid_df = train_test_split(balanced_train_df, 
                 test_size = 0.2,
                 stratify = balanced_train_df['image_group'])


train_df = shuffle(train_df) # Shuffle since some images could be grouped
print('Training len: ', train_df.shape[0], "  Validation len: ", valid_df.shape[0])

# set lengths and steps
train_len = len(train_df)
val_len = len(valid_df)
images_list_len = train_len + val_len

steps_per_epoch = np.ceil(train_len // parms.BATCH_SIZE) # set step sizes based on train & batch
validation_steps = np.ceil(val_len // parms.BATCH_SIZE) # set step sizes based on val & batch

print("Total number: ", images_list_len, "  Train number: ", train_len, "  Val number: ", val_len)
print("Steps/EPOC: ", steps_per_epoch, "  Steps/Validation: ", validation_steps)


## Build, load and augment TensorFlow Datasets

In [0]:
# Aug image and mask
def image_mask_aug(image: tf.Tensor, mask: tf.Tensor) -> tf.Tensor:
    
    if tf.random.uniform(()) > 0.0:     #.25, .1
        k = tf.random.uniform(shape=[], minval=1, maxval=4, dtype=tf.int32)
        image = tf.image.rot90(image, k) #0-4, 0/360, 90/180/270
        mask = tf.image.rot90(mask, k) #0-4, 0/360, 90/180/270

    if tf.random.uniform(()) > 0.25:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
        
    if tf.random.uniform(()) > 0.25:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)

    # Really rough way to adjust the image, larger gamma => darken image, smaller gamma => lightens image
    # so use image mean and add 0.5 to get values between 05 and 1.5, apply those
    # using adjust_gamma.  (image mean of 0 would be a black image, mean of 255 would be a white image)
    # If applied to all, then should also apply to testing data
    gamma = tf.math.reduce_mean(image) + 0.5
    image = tf.image.adjust_gamma(image, gamma=gamma)
    ###########################################################################

    # these also help training, but the gamma worked a little better, can play around with both
    #if tf.random.uniform(()) > 0.5: 
    #    image = tf.image.adjust_brightness(image, delta=0.2)

    #if tf.random.uniform(()) > 0.5: 
    #    image = tf.image.adjust_contrast(image, contrast_factor=0.2)

    return image, mask

def load_image_mask(image_id: tf.Tensor) -> tf.Tensor:
    # load image from file name
    file_path = parms.TRAIN_PATH + "/images/" + image_id
    # load the raw data from the file as a string
    image = tf.io.read_file(file_path)
    # convert the png compressed string to a 3D tensor
    image = tf.image.decode_png(image, channels=parms.IMAGE_CHANNELS)
    image = tf.image.convert_image_dtype(image, parms.IMAGE_DTYPE)

    # load the mask
    file_path = parms.TRAIN_PATH + "/masks/" + image_id
    # load the raw data from the file as a string
    mask_orig = tf.io.read_file(file_path)
    # convert the compressed string to a 3D uint8 tensor
    mask_orig = tf.image.decode_png(mask_orig, channels=1)
    # Convert to 1 or 0
    mask_adj = tf.where(mask_orig > 0, 1, 0)
    mask_adj = tf.cast(mask_adj, dtype=tf.float32)

    # actual mask - used for model verification, ignored for training
    mask_orig = tf.where(mask_orig > 0, 1, 0)
    mask_orig = tf.cast(mask_orig, dtype=tf.float32)

    return image, mask_adj, mask_orig

def image_mask_resize(image, mask):
    image = tf.image.resize(image, [parms.IMAGE_ROWS, parms.IMAGE_COLS]) 
    mask = tf.image.resize(mask, [parms.IMAGE_ROWS, parms.IMAGE_COLS]) 
    return image, mask

# mapped method to load image and mask
def process_train_image_id(image_id: tf.Tensor) -> tf.Tensor:
    image, mask_adj, mask_orig = load_image_mask(image_id)
    image, mask_adj = image_mask_aug(image, mask_adj)

    # You can resize then augment, or augment then resize
    # Since this enlarges the image, I like to do the augmentation first, then resize
    image, mask_adj = image_mask_resize(image, mask_adj)

    return image, mask_adj

def process_val_image_id(image_id: tf.Tensor) -> tf.Tensor:
    image, mask_adj, mask_orig = load_image_mask(image_id)

    # did not apply gamma just to see if any difference...
    #gamma = tf.math.reduce_mean(image) + 0.5
    #image = tf.image.adjust_gamma(image, gamma=gamma)
    ###########################################################################

    image, mask_adj = image_mask_resize(image, mask_adj)

    return image, mask_adj


In [0]:
# Create Dataset from pf
train_dataset = tf.data.Dataset.from_tensor_slices(train_df["ImageId"].values)

# Verify image paths were loaded
for image_id in train_dataset.take(2):
    print("Image id: ", image_id.numpy().decode("utf-8"))

# map training images to processing, includes any augmentation
train_dataset = train_dataset.map(process_train_image_id, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, mask in train_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Encoded Pixels shape: ", mask.numpy().shape, type(mask.numpy()[0][0][0]))
    some_image = image.numpy()
    some_mask = mask.numpy()

#show_batch_mask([some_image, some_mask])

train_dataset = train_dataset.cache().batch(parms.BATCH_SIZE).repeat()

# Show the images, execute this cell multiple times to see the images
for image, mask in train_dataset.take(1):
    sample_image, sample_mask = image[0], mask[0]
show_batch_mask([sample_image, sample_mask])

In [0]:
# Create Dataset from pd
val_dataset = tf.data.Dataset.from_tensor_slices(valid_df["ImageId"].values)

# Verify image paths were loaded
for image_id in val_dataset.take(2):
    print("Image id: ", image_id.numpy().decode("utf-8"))

    # map training images to processing, includes any augmentation
val_dataset = val_dataset.map(process_val_image_id, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, mask in val_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("Encoded Pixels shape: ", mask.numpy().shape)
    some_image = image.numpy()
    some_mask = mask.numpy()

#show_batch_mask([some_image, some_mask])
#val_dataset = val_dataset.cache().batch(parms.BATCH_SIZE).repeat()  # uncomment if there is enough memory, speeds up training

val_dataset = val_dataset.batch(parms.BATCH_SIZE).repeat()

In [0]:
# Final check before model training.  I added a string of the mask non-zero counts - need to make sure the masks 
# were created ok.  (got bit by this one after a small change....)

# Test Validation or Train by changing the dataset

mask_cnt_str = ""
sample_image = None
sample_mask = None
#for image, mask in train_dataset.take(1):
for image, mask in val_dataset.take(1):
    image_np = image.numpy()
    mask_np = mask.numpy()
    for i in range(len(image_np)):
        # uncomment to show actual batch of images for training or validation
        #show_batch_mask([image[i], mask[i]])  # Will show all of the batch
        mask_cnt_str = mask_cnt_str + str(np.count_nonzero(mask_np[i])) + "  "
        #print("Mask shape: {}  Max: {}  Min: {}".format(mask.numpy().shape, np.max(mask.numpy()), np.min(mask.numpy())))

        if np.count_nonzero(mask_np[i]) > 0:
            sample_image, sample_mask = image[i], mask[i]
            
print("Mask counts: ", mask_cnt_str)
show_batch_mask([sample_image, sample_mask])  # Will show the sample masks, if errors, then no mask was found content

In [0]:
# If you want to see the improvements after each EPOC, add to the callback. Helps to make sure show_predictions works...
# Forked from Segmentation example
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

# Normal callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
reduce_lr = ReduceLROnPlateau(monitor='val_lb_metric', factor=0.33, patience=4, verbose=1, mode='max', min_delta=0.0001, 
                              cooldown=0, min_lr=1e-8)
earlystopper = EarlyStopping(monitor="val_lb_metric", mode="max", verbose=2, patience=12)
checkpointer = ModelCheckpoint(parms.MODEL_PATH, monitor='val_lb_metric', verbose=1, mode="max", save_best_only=True)
  
# Methods to support training verification
def create_mask(pred_mask):
    pred_mask = np.where(pred_mask > 0.5, 1, 0)
    return pred_mask[0]

# Shows the image, original mask and predicted mask
def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            show_batch_mask([image[0], mask[0], create_mask(pred_mask)])
    else:
        show_batch_mask([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

# Forked from https://github.com/ybabakhin/kaggle_salt_bes_phalanx
loss_function = "bce_dice"  # bce_dice, lovasz
def loss(y, p):
    return dice_coef_loss_bce(y, p, dice=0.5, bce=0.5)

def lb_metric(y_true, y_pred):
    return Kaggle_IoU_Precision(y_true, y_pred, threshold=0 if loss_function == 'lovasz' else 0.5)
    
def compile_model(parms, model):
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                  loss= make_loss(loss_function),
                  metrics=[lb_metric])

    return model


In [0]:
# Reload the model from prior runs
#model = load_model(parms.MODEL_PATH, custom_objects={'loss': loss, 'lb_metric': lb_metric})


In [0]:
# train from empty seg model, comment if loading existing model
model = load_model(STARTING_MODEL_PATH)

model = compile_model(parms, model)

In [0]:
# uncomment to draw the model...
#tf.keras.utils.plot_model(model, show_shapes=True)

In [0]:
# Train
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=parms.EPOCS, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[reduce_lr, earlystopper, checkpointer])


In [0]:
# graph training...
history_df = pd.DataFrame(history.history)
plt.figure()
history_df[['loss', 'val_loss']].plot(title="Loss")
plt.xlabel('Epocs')
plt.ylabel('Loss')
history_df[['lb_metric', 'val_lb_metric']].plot(title="Kaggle LB")
plt.xlabel('Epocs')
plt.ylabel('Accuracy')

plt.show()

# Validate the training...

- Show images and predictions
- Try and improve the prediction by cleaning up the mask

In [0]:
# Reload the model from prior run
model = load_model(parms.MODEL_PATH, custom_objects={'loss': loss, 'lb_metric': lb_metric})


In [0]:
# Easy to modify to predict the given test files that do not have a mask

# Method to be applied to all testing images
def process_test_image_id(image_id: tf.Tensor) -> tf.Tensor:
    image, mask_adj, mask_orig = load_image_mask(image_id)  

    # since applied to all iamges, apply for testing validation
    gamma = tf.math.reduce_mean(image) + 0.5
    image = tf.image.adjust_gamma(image, gamma=gamma)
    ###########################################################
    
    image, mask_adj = image_mask_resize(image, mask_adj)

    return image_id, image, mask_adj, mask_orig

# Create Dataset from pd, could use validation or training
test_df = shuffle(valid_df)
test_dataset = tf.data.Dataset.from_tensor_slices(test_df["ImageId"].values)

# Verify image paths were loaded
for image_id in test_dataset.take(2):
    print(image_id.numpy().decode("utf-8"))

    # map training images to processing, includes any augmentation
test_dataset = test_dataset.map(process_test_image_id, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image_id, image, mask_adj, mask_orig in test_dataset.take(1):
    print("Image Id: ", image_id.numpy().decode("utf-8"))
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("(128, 128) mask shape: ", mask_adj.numpy().shape)
    print("(101, 101) mask shape: ", mask_orig.numpy().shape)    
    some_image = image.numpy()
    some_mask = mask_adj.numpy()
    
test_dataset = test_dataset.batch(1).repeat()

In [0]:
from skimage.morphology import erosion, dilation, disk, flood_fill

def mask_erosion_dilation(mask, disksize=6):
    """
    # ONLY GRAYSCALE
    erode and dialate an image - cleans up random pixels
    opening operation. The purpose of this operation is to remove small
    islands of noise while (trying to) maintain the areas of the larger
    objects in your image
    """
    #print("ero ", mask.shape)
    mask = mask.reshape((mask.shape[0], mask.shape[1]))
    selem = disk(disksize)
    mask = dilation(erosion(mask, selem), selem)
    return mask.reshape((mask.shape[0], mask.shape[1], 1))

def show_batch_mask_aug(image_id, display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask', "Aug Mask", "Adj to 101"]

    for i in range(len(display_list)):
        #print(np.max(display_list[i]), np.min(display_list[i]))
        plt.subplot(1, len(display_list), i+1)
        if i == 0:
            plt.title(image_id)
        else:
            plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]),cmap='gray')
        plt.axis('off')
    plt.show()

def test_model_with_aug(dataset, model, steps, test=False):
    if test and steps > 30:
        print("CHANGE STEPS, TOO LARGE.... ", steps)
        return

    kaggle_percision_adj = 0.0
    kaggle_percision_orig = 0.0
    for image_id, image, mask_adj, mask_orig in tqdm(dataset.take(steps)):

        pred_mask = model.predict(image)
        mask_actual = create_mask(pred_mask)
         
        mask_aug = mask_erosion_dilation(mask_actual)
        
        # Score based on (224, 224) size
        kaggle_percision_adj_tmp = Kaggle_IoU_Precision(mask_adj, np.expand_dims(mask_aug, axis=0)).numpy()
        kaggle_percision_adj += kaggle_percision_adj_tmp

        # Score based on (101, 101) size - Double check score after reducing
        mask_aug_adj = tf.image.resize(mask_aug, [101, 101])
        kaggle_percision_orig_tmp = Kaggle_IoU_Precision(mask_orig, np.expand_dims(mask_aug_adj, axis=0)).numpy()
        kaggle_percision_orig += kaggle_percision_orig_tmp

        if test:
            img_id = image_id.numpy()[0].decode("utf-8")
            print("true vs true: ", Kaggle_IoU_Precision(mask_orig, mask_orig).numpy())
            print("true vs pred: ", Kaggle_IoU_Precision(mask_adj, np.expand_dims(mask_actual, axis=0)).numpy())
            print("true vs aug: ", Kaggle_IoU_Precision(mask_adj, np.expand_dims(mask_aug, axis=0)).numpy())
            print("101 true vs 101 aug: ", Kaggle_IoU_Precision(mask_orig, np.expand_dims(mask_aug_adj, axis=0)).numpy())
            print("non-zero counts,  act: ", np.count_nonzero(mask_adj), "  pred: ", np.count_nonzero(mask_actual), "  aug: ", np.count_nonzero(mask_aug))
            print("101 non-zero counts,  act: ", np.count_nonzero(mask_orig), "  pred: ", np.count_nonzero(mask_aug_adj))
            show_batch_mask_aug(img_id, [image[0], mask_adj[0], mask_actual, mask_aug, mask_aug_adj])

    kaggle_percision_adj = kaggle_percision_adj / steps
    kaggle_percision_orig = kaggle_percision_orig / steps
    return kaggle_percision_adj, kaggle_percision_orig


In [0]:
kaggle_percision_adj, kaggle_percision_orig = test_model_with_aug(test_dataset, model, 1395, test=False) # 5579 training  1395 validation
print(" ")
print("   (224, 224) Score: ", kaggle_percision_adj, "   (101, 101) Score: ", kaggle_percision_orig)